# Anomaly Detection using Anomalib library 

## PatchCore Alogrithm

In [ ]:
# pip install --upgrade --force-reinstall pip==24.0
# pip install -r requirements.txt
# pip install anomalib==1.0.1

In [ ]:
!pip --version
!python --version

 ### Import libraries

In [ ]:
from anomalib.data.utils import ValSplitMode
from anomalib.deploy import ExportType
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from anomalib.models import Patchcore, Padim, EfficientAd
from anomalib import TaskType
from anomalib.data.image.folder import Folder
from anomalib.loggers import AnomalibWandbLogger
from anomalib.engine import Engine
from torchvision.transforms import v2
from torchvision.transforms.v2.functional import to_pil_image
import os

transforms = None

 ### Load Dataset

In [ ]:
dataset_root = "MVTecAD"
print(f"Dataset root: {dataset_root}")
name = 'MVTecAD-hazelnut'
patience = 3

datamodule = Folder(
    name=name,
    root=dataset_root,
    normal_dir="normal",
    abnormal_dir="abnormal",
    normal_test_dir="normal-test",
    num_workers=4,
    task=TaskType.CLASSIFICATION,
    seed=42,
    # test_split_ratio=0.2,
    val_split_mode=ValSplitMode.FROM_TEST, # default value
    val_split_ratio=0.5, # default value
    train_transform=transforms
)
datamodule.setup()

In [ ]:
# Train images
i, data = next(enumerate(datamodule.train_dataloader()))
print(data.keys(), data["image"].shape)

for i, data in enumerate(datamodule.train_dataloader()):
    print(i, len(data["image_path"]))

In [ ]:
for i, data in enumerate(datamodule.test_dataloader()):
    
    print(i, len(data["image_path"]))
    # for i in range(len(data["image_path"])):
    #     print(data["image_path"][i][39:], '-', data["label"][i], data["image"][i].shape)

In [ ]:
# from the datamodule extract the train, val and test Pandas dataset and collect all the info in a csv
train_dataset = datamodule.train_data.samples
test_dataset = datamodule.test_data.samples
val_dataset = datamodule.val_data.samples

# check the data distribution for each category in each data split
print("TRAIN DATASET FEATURES")
print(train_dataset.info())
print("")
print("IMAGE DISTRIBUTION BY CLASS")
# print("")
desc_grouped = train_dataset[['label']].value_counts()
print(desc_grouped)
print('\n')
print("----------------------------------------------------------")
print("TEST DATASET FEATURES")
print(test_dataset.info())
print("")
print("IMAGE DISTRIBUTION BY CLASS")
# print("")
desc_grouped = test_dataset[['label']].value_counts()
print(desc_grouped)
print('\n')
print("----------------------------------------------------------")
print("VAL DATASET FEATURES")
print(val_dataset.info())
print("")
print("IMAGE DISTRIBUTION BY CLASS")
print("")
desc_grouped = val_dataset[['label']].value_counts()
print(desc_grouped)

# datamodule.train_data.samples.to_csv(os.path.join("/kaggle/working/", "datamodule_train.csv"), index=False)
# datamodule.test_data.samples.to_csv(os.path.join("/kaggle/working/", "datamodule_test.csv"), index=False)
# datamodule.val_data.samples.to_csv(os.path.join("/kaggle/working/", "datamodule_val.csv"), index=False)

 ### Model training

In [ ]:
model = Patchcore(coreset_sampling_ratio=0.1, num_neighbors=9)
# model = Padim()
# model = EfficientAd()

callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="image_AUROC",
        save_last=True,
        verbose=True,
        auto_insert_metric_name=True,
        every_n_epochs=1,
    ),
    EarlyStopping(
            monitor="image_AUROC",
            mode="max",
            patience=patience,
        ),
]


engine = Engine(
    # callbacks=callbacks,
    # threshold='F1AdaptiveThreshold',
    # image_metrics='AUROC',
    # pixel_metrics="AUROC",
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    task=TaskType.CLASSIFICATION,
)

print("Fit...")
engine.fit(datamodule=datamodule, model=model)

print("Test...")
engine.test(datamodule=datamodule, model=model)

# Export in torch
print("Export weights...")
path_export_weights = engine.export(export_type=ExportType.TORCH,
                                    model=model)

print("path_export_weights: ", path_export_weights)